In [1]:
import numpy as np
import pandas as pd
import os
import os.path as osp
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import random
import copy
import time
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold

from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau
import torch.utils.data as data
from torch.cuda.amp import autocast, GradScaler
import torchvision
from torchvision import models, transforms
import timm
import gc


# from bi_tempered_loss_pytorch import bi_tempered_logistic_loss
# from temperature_scaling import ModelWithTemperature

#import lightgbm as lgb
import pickle
start = time.time()

In [2]:
config = {
    'rootpath': osp.join('..',"..",'input','cassava-leaf-disease-classification'),
    'seed': 334,
    'num_folds': 5,
    
    # augmentaion params
    'aug': {
        'ver': 'V3',
        'size': 512,
#         'size': 384,
#         'size': 224,
        'mean': (0.485, 0.456, 0.406),
        'std': (0.229, 0.224, 0.225),
#         'degrees': 45,
        'degrees': 90,
        'brightness': 0.3,
        'contrast': 0.3,
    },
    
    'do_mix': False,
    'do_TemperatureScaling': False,
    'freezeBN': True,
    
    # loss params
    'loss': {
#         'name': 'CrossEntropyLoss',
        'name': 'BiTemperedLogisticLoss',
        't1': 0.2,
        't2': 1.2,
#         't2': 4.0,
#         'label_smoothing': 0.01,
        'label_smoothing': 0.05,
        'num_iters': 5,
        'reduction': 'mean'
    },

#     'model_name': 'resnet50',
#     'model_name': 'resnext50_32x4d',
#     'model_name': 'seresnext50_32x4d',
    'model_name': 'tf_efficientnet_b4_ns',
    'batch_size': 8,
    'test_batch_size': 1,
    'num_epochs': 10,
    'set_all': True,
    
    # optim params
    'optim': {
#         'name': 'SGD',
        'name': 'Adam',
        'lr': 1e-04,
        'T_0': 10,   # CosineAnnealingWarmRestarts
        'T_mult': 1,   # CosineAnnealingWarmRestarts
        'min_lr': 1e-06,   # CosineAnnealingWarmRestarts
        'last_epoch': -1,   # CosineAnnealingWarmRestarts
        'weight_decay': 1e-06
    },
    
    'tta': {
        #'num': 8
        'num': 5,
    }
}

In [3]:
class ImageTransform():
    def __init__(self, resize, mean, std, degrees=45, brightness=0.3, contrast=0.3):
        self.data_transform = {
            'V1': {
                'train': transforms.Compose([
                    transforms.RandomResizedCrop(resize, scale=(0.5, 1.0)),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Normalize(mean, std)
                ]),
                'val': transforms.Compose([
                    transforms.Resize(resize),
                    transforms.CenterCrop(resize),
                    transforms.ToTensor(),
                    transforms.Normalize(mean, std)
                ]),
                'test': transforms.Compose([
                    transforms.Resize(resize),
                    transforms.CenterCrop(resize),
                    transforms.ToTensor(),
                    transforms.Normalize(mean, std)
                ])                
            },
            'V2': {
                'train': transforms.Compose([
                    transforms.RandomResizedCrop(resize, scale=(0.5, 1.0)),
                    transforms.RandomHorizontalFlip(),
                    transforms.RandomVerticalFlip(),
                    transforms.RandomRotation(degrees),
                    transforms.ColorJitter(brightness, contrast),
                    transforms.ToTensor(),
                    transforms.Normalize(mean, std)
                ]),
                'val': transforms.Compose([
                    transforms.Resize(resize),
                    transforms.CenterCrop(resize),
                    transforms.ToTensor(),
                    transforms.Normalize(mean, std)
                ]),
                'test': transforms.Compose([
                    transforms.Resize(resize),
                    transforms.CenterCrop(resize),
                    transforms.ToTensor(),
                    transforms.Normalize(mean, std)
                ])
            },
            'V3': {
                'train': transforms.Compose([
                    transforms.RandomResizedCrop(resize),
                    transforms.RandomHorizontalFlip(),
                    transforms.RandomVerticalFlip(),
                    transforms.RandomRotation(degrees),
                    transforms.ColorJitter(brightness, contrast),
                    transforms.ToTensor(),
                    transforms.Normalize(mean, std)
                ]),
                'val': transforms.Compose([
                    transforms.Resize(resize),
                    transforms.CenterCrop(resize),
                    transforms.ToTensor(),
                    transforms.Normalize(mean, std)
                ]),
                'test': transforms.Compose([
                    transforms.Resize(resize),
                    transforms.CenterCrop(resize),
                    transforms.ToTensor(),
                    transforms.Normalize(mean, std)
                ])
            }
        }

    def __call__(self, img, ver='V1', phase='train'):
        return self.data_transform[ver][phase](img)

In [4]:
class CassavaDataset(data.Dataset):
    def __init__(self, filepath2label, transform=None, ver='V1', phase='train', output_label=True):
        self.file_list = list(filepath2label.keys())
        self.transform = transform
        self.filepath2label = filepath2label
        self.ver = ver
        self.phase = phase
        self.output_label = output_label
        
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, index):
        img_path = self.file_list[index]
        img = Image.open(img_path)
        if self.transform:
            img = self.transform(img, self.ver, self.phase)
        if self.output_label:
            label = self.filepath2label[img_path]
            return img, label
        else:
            return img        

In [5]:
def get_DataLoader(dataset, batch_size, shuffle=True):
    return torch.utils.data.DataLoader(dataset, batch_size, shuffle)

In [6]:
class CustomResNext(nn.Module):
    def __init__(self, model_name='resnext50_32x4d', pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        n_features = self.model.fc.in_features
        self.model.fc = nn.Linear(n_features, 5)

    def forward(self, x):
        x = self.model(x)
        return x

In [7]:
class CustomEfficientNet(nn.Module):
    def __init__(self, model_name='tf_efficientnet_b4_ns', pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, 5)
        '''
        self.model.classifier = nn.Sequential(
            nn.Dropout(0.3),
            #nn.Linear(n_features, hidden_size,bias=True), nn.ELU(),
            nn.Linear(n_features, n_class, bias=True)
        )
        '''
    def forward(self, x):
        x = self.model(x)
        return x

In [8]:
def get_network(model_name, use_pretrained=True):
    if model_name == 'vgg19':
        net = models.vgg19(pretrained=use_pretrained)
        net.classifier[6] = nn.Linear(in_features=4096, out_features=5, bias=True)
        update_param_names = ['classifier.6.weight', 'classifier.6.bias']
    elif model_name == 'resnet50':
        net = models.resnet50(pretrained=use_pretrained)
        net.fc = nn.Linear(in_features=2048, out_features=5, bias=True)
        update_param_names = ['fc.weight', 'fc.bias']
    elif model_name == 'resnext50_32x4d':
        net = CustomResNext(model_name=model_name, pretrained=use_pretrained)
        update_param_names = ['model.fc.weight', 'model.fc.bias']
    elif model_name == 'seresnext50_32x4d':
        net = CustomResNext(model_name=model_name, pretrained=use_pretrained)
        update_param_names = ['model.fc.weight', 'model.fc.bias']
    elif model_name == 'tf_efficientnet_b5_ns':
        net = CustomEfficientNet(model_name=model_name, pretrained=use_pretrained)
        update_param_names = ['model.classifier.weight', 'model.classifier.bias']
    elif model_name == 'tf_efficientnet_b4_ns':
        net = CustomEfficientNet(model_name=model_name, pretrained=use_pretrained)
        update_param_names = ['model.classifier.weight', 'model.classifier.bias']
    elif model_name == 'tf_efficientnet_b3_ns':
        net = CustomEfficientNet(model_name=model_name, pretrained=use_pretrained)
        update_param_names = ['model.classifier.weight', 'model.classifier.bias']
    elif model_name == 'tf_efficientnet_b1_ns':
        net = CustomEfficientNet(model_name=model_name, pretrained=use_pretrained)
        update_param_names = ['model.classifier.weight', 'model.classifier.bias']
        
    return net, update_param_names


In [9]:
def inference_test_data_tta(net, dataloader, device):
    net.eval()
    preds = []
#     bar = tqdm(dataloader)
    
    with torch.no_grad():
        for inputs in dataloader:
            inputs = inputs.to(device)
            if config['tta']['num'] == 8:
                inputs = torch.stack([inputs, inputs.flip(-1), inputs.flip(-2), inputs.flip(-1,-2),
                                      inputs.transpose(-1,-2), inputs.transpose(-1,-2).flip(-1), 
                                      inputs.transpose(-1,-2).flip(-2), inputs.transpose(-1,-2).flip(-1,-2)], 0)
            elif config['tta']['num'] == 5:
                inputs = torch.stack([inputs, inputs.flip(-1), inputs.flip(-2), inputs.flip(-1,-2),
                                      inputs.transpose(-1,-2)], 0)
            inputs = inputs.view(-1, 3, config['aug']['size'], config['aug']['size'])
            logits = net(inputs)
            logits = logits.view(config['test_batch_size'], config['tta']['num'], -1).mean(1)
            preds += [torch.softmax(logits, 1).detach().cpu()]
        preds = torch.cat(preds).cpu().numpy()
    return preds

In [10]:
def set_random_seed(seed):
    # set random seed
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [11]:
def make_datapath_list(rootpath, phase='train'):
    target_path = osp.join(rootpath, phase+'_images', '*.jpg')

    path_list = []
    for path in glob.glob(target_path):
        path_list.append(path)
    return path_list

In [12]:
def make_filename_info(title, ext, model_name, fold, aug_ver, loss_name, num_epochs, batch_size, size, set_all, lr, do_mix, freezeBN):
    if loss_name == 'CrossEntropyLoss':
        filename = title+'_'+model_name+'_fold'+str(fold)+'_'+aug_ver+'_loss'+loss_name+'_epoch'+str(num_epochs)+'_batchsize'+str(batch_size)+'_imgsize'+str(size)+'_allparams'+str(set_all)+'_lr'+str(lr)+'_Mix'+str(do_mix)+'_freezeBN'+str(freezeBN)+ext
    elif loss_name == 'BiTemperedLogisticLoss':
        filename = title+'_'+model_name+'_fold'+str(fold)+'_'+aug_ver+'_loss'+loss_name+'_t1'+str(config['loss']['t1'])+'_t2'+str(config['loss']['t2'])+'_labelsmoothing'+str(config['loss']['label_smoothing'])+'_epoch'+str(num_epochs)+'_batchsize'+str(batch_size)+'_imgsize'+str(size)+'_allparams'+str(set_all)+'_lr'+str(lr)+'_Mix'+str(do_mix)+'_freezeBN'+str(freezeBN)+ext
    return filename

In [13]:
resnet50_list = np.sort(glob.glob(os.path.join('..\\..\\input\\stacking_ensemble_data\\resnet50',"*.pth")))
efnetb1_list = np.sort(glob.glob(os.path.join('..\\..\\input\\stacking_ensemble_data\\tf_efficientnet_b1_ns',"*.pth")))
efnetb4_list = np.sort(glob.glob(os.path.join('..\\..\\input\\stacking_ensemble_data\\tf_efficientnet_b4_ns',"*.pth")))
efnetb5_list = np.sort(glob.glob(os.path.join('..\\..\\input\\stacking_ensemble_data\\tf_efficientnet_b5_ns',"*.pth")))

In [14]:
set_random_seed(config['seed'])

# device setting
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("use device: ", device)

#test_datalist = np.sort(make_datapath_list(config['rootpath'], 'test'))
test_datalist = np.sort(make_datapath_list(config['rootpath'], 'train'))[:100]

test_filepath2label = dict(zip(test_datalist, [0] * len(test_datalist)))
test_dataset = CassavaDataset(test_filepath2label, ImageTransform(config['aug']['size'], config['aug']['mean'], config['aug']['std'], config['aug']['degrees'], config['aug']['brightness'], config['aug']['contrast']), config['aug']['ver'], 'test', False)
test_dataloader = get_DataLoader(test_dataset, config['test_batch_size'], False)
test_preds = []

inputs_metamodel = np.array([])


feature_col = []
#########################################
### TTA pred resnet50 ###
#########################################
test_preds = []
model_list = resnet50_list
feature_col += ["resnet50-c%d_proba"%i for i in range(5)]
for load_path in model_list:
    print(load_path)
    load_weights = torch.load(load_path, map_location=device)
    net, update_param_names = get_network('resnet50', False)
    net.load_state_dict(load_weights)
    net.to(device)
    test_preds += [inference_test_data_tta(net, test_dataloader, device)]

if inputs_metamodel.size == 0:
    inputs_metamodel = np.mean(test_preds, axis=0)
else:
    inputs_metamodel = np.hstack([inputs_metamodel, np.mean(test_preds, axis=0)])



#########################################
### TTA pred efficient net b1 ###
#########################################
test_preds = []
feature_col += ["tf_efficientnet_b1_ns-c%d_proba"%i for i in range(5)]
model_list = efnetb1_list
for load_path in model_list:
    print(load_path)
    load_weights = torch.load(load_path, map_location=device)
    net, update_param_names = get_network('tf_efficientnet_b1_ns', False)
    net.load_state_dict(load_weights)
    net.to(device)
    test_preds += [inference_test_data_tta(net, test_dataloader, device)]

if inputs_metamodel.size == 0:
    inputs_metamodel = np.mean(test_preds, axis=0)
else:
    inputs_metamodel = np.hstack([inputs_metamodel, np.mean(test_preds, axis=0)])


#########################################
### TTA pred efficient net b4 ###
#########################################
test_preds = []
feature_col += ["tf_efficientnet_b4_ns-c%d_proba"%i for i in range(5)]
model_list = efnetb4_list
for load_path in model_list:
    print(load_path)
    load_weights = torch.load(load_path, map_location=device)
    net, update_param_names = get_network('tf_efficientnet_b4_ns', False)
    net.load_state_dict(load_weights)
    net.to(device)
    test_preds += [inference_test_data_tta(net, test_dataloader, device)]

if inputs_metamodel.size == 0:
    inputs_metamodel = np.mean(test_preds, axis=0)
else:
    inputs_metamodel = np.hstack([inputs_metamodel, np.mean(test_preds, axis=0)])


#########################################
### TTA pred efficient net b4 ###
#########################################
test_preds = []
feature_col += ["tf_efficientnet_b5_ns-c%d_proba"%i for i in range(5)]
model_list = efnetb5_list
for load_path in model_list:
    print(load_path)
    load_weights = torch.load(load_path, map_location=device)
    net, update_param_names = get_network('tf_efficientnet_b5_ns', False)
    net.load_state_dict(load_weights)
    net.to(device)
    test_preds += [inference_test_data_tta(net, test_dataloader, device)]

if inputs_metamodel.size == 0:
    inputs_metamodel = np.mean(test_preds, axis=0)
else:
    inputs_metamodel = np.hstack([inputs_metamodel, np.mean(test_preds, axis=0)])


    
del net, load_weights, test_preds, test_dataloader
gc.collect()

use device:  cuda
..\..\input\stacking_ensemble_data\resnet50\model_resnet50_fold0.pth
..\..\input\stacking_ensemble_data\resnet50\model_resnet50_fold1.pth
..\..\input\stacking_ensemble_data\resnet50\model_resnet50_fold2.pth
..\..\input\stacking_ensemble_data\resnet50\model_resnet50_fold3.pth
..\..\input\stacking_ensemble_data\resnet50\model_resnet50_fold4.pth
..\..\input\stacking_ensemble_data\tf_efficientnet_b1_ns\model_tf_efficientnet_b1_ns_fold0.pth
..\..\input\stacking_ensemble_data\tf_efficientnet_b1_ns\model_tf_efficientnet_b1_ns_fold1.pth
..\..\input\stacking_ensemble_data\tf_efficientnet_b1_ns\model_tf_efficientnet_b1_ns_fold2.pth
..\..\input\stacking_ensemble_data\tf_efficientnet_b1_ns\model_tf_efficientnet_b1_ns_fold3.pth
..\..\input\stacking_ensemble_data\tf_efficientnet_b1_ns\model_tf_efficientnet_b1_ns_fold4.pth
..\..\input\stacking_ensemble_data\tf_efficientnet_b4_ns\model_tf_efficientnet_b4_ns_fold0.pth
..\..\input\stacking_ensemble_data\tf_efficientnet_b4_ns\model_tf_e

0

In [20]:
sub_df = pd.DataFrame(index = [osp.basename(k) for k in test_datalist],
                      columns = feature_col,
                      data = inputs_metamodel)
sub_df["label"] = 0
sub_df.index.name = "image_id"

In [21]:
from torch.utils.data import DataLoader, Dataset
class StackingDataset(data.Dataset):
    def __init__(self, df,features,target):
        self.df = df
        self.target = target
        self.features = features
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        
        label = self.df[self.target].values[index]
        img = np.array(self.df[self.features].iloc[index])
        img = img.reshape([-1,5])[np.newaxis,:,:]
        img = torch.from_numpy(img)

        return img.float(), int(label)

test_dataloader = DataLoader(StackingDataset(sub_df,feature_col,"label"),
                              batch_size=1,
                              shuffle=False)

In [22]:
class my2DCNN(nn.Module):
    def __init__(self,model_num):
        super().__init__()
        
        self.feature = nn.Sequential(
            nn.Conv2d(1,8,kernel_size=(1,3),stride=1,padding=0),
            nn.ReLU(inplace=True),
            nn.Conv2d(8,16,kernel_size=(1,3),stride=1,padding=0),
            nn.ReLU(inplace=True)
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(16*model_num*1,2048),
            nn.ReLU(inplace=True),
            nn.Dropout(0.8),
            nn.Linear(2048,5)
        )
        
        self.model_num = model_num
        
        
    def forward(self,x):
        x = self.feature(x)
        x = x.view(-1,16*self.model_num*1)
        x = self.classifier(x)
        
        return x
    
    def check_cnn_size(self,x):
        out = self.feature(x)
        return out

meta_model_path = "2D_CNN-Adam(lr0.0001)-BiTLLoss(0.2,1.2)-epoch56.pth"
meta_model = my2DCNN(model_num=4)
meta_model.load_state_dict(torch.load(meta_model_path))
print(meta_model)

my2DCNN(
  (feature): Sequential(
    (0): Conv2d(1, 8, kernel_size=(1, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(8, 16, kernel_size=(1, 3), stride=(1, 1))
    (3): ReLU(inplace=True)
  )
  (classifier): Sequential(
    (0): Linear(in_features=64, out_features=2048, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.8, inplace=False)
    (3): Linear(in_features=2048, out_features=5, bias=True)
  )
)


In [23]:
cols = ["label"]
    
meta_model.eval()
meta_model = meta_model.to(device)
outputs = []

for batch in tqdm(test_dataloader):

    X = batch[0].to(device)
    y = batch[1].to(device)

    pred = meta_model(X)
    pred = torch.nn.Softmax(dim=1)(pred)
    pred = pred.cpu().detach().numpy()
    pred = pred.argmax(axis=1)

    pred = pd.DataFrame(columns=cols,
                         data=pred)

    outputs.append(pred)
outputs = pd.concat(outputs)
outputs = outputs.reset_index(drop=True)

sub_df = sub_df.reset_index()
sub_df = sub_df.drop(columns={"label"},axis=15)

sub_df = pd.concat( [sub_df, outputs], axis=1 )
sub_df = sub_df[["image_id","label"]]
sub_df.to_csv("submission.csv",index=False)

display(sub_df)

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 659.66it/s]


,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,4
3,1000723321.jpg,1
4,1000812911.jpg,3
...,...,...
95,1016334938.jpg,3
96,1016415263.jpg,1
97,10169000.jpg,1
98,1016955090.jpg,2
